In [23]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests
import numpy as np
import ipeadatapy as ip
from os.path import exists


# get data from ipea or reads from file
def get_ipea_data():
    if not exists("data/petr_brent.csv"):

        # baixa os dados do site do ipea
        # ip_metadata = ip.metadata()
        # busca do código da série de Preço - petróleo bruto - Brent (FOB)
        # ip_metadata[ip_metadata["NAME"].str.contains("Brent") == True]

        # seleção da série com dados a partir de 2001
        petr_brent = ip.timeseries("EIA366_PBRENT366", yearGreaterThan=2000)
        petr_brent.to_csv("data/petr_brent.csv")
    else:
        petr_brent = pd.read_csv("data/petr_brent.csv")

    return petr_brent


# load data and fill null values
def load_data():

    # get data from ipea or reads from file
    petr_brent = get_ipea_data()

    # print(petr_brent.head())

    # convert RAW DATE to datetime
    petr_brent["DATE"] = pd.to_datetime(petr_brent["DATE"])

    # select only DATE and VALUE (US$)
    petr_brent = petr_brent[["DATE", "VALUE (US$)"]]

    petr_brent.set_index("DATE", inplace=True)

    # select the first and last date
    start = petr_brent.index[0].date()
    end = petr_brent.index[len(petr_brent) - 1].date()
    new_dates = pd.date_range(start=start, end=end, freq="D")

    # reindex dates
    petr_brent = petr_brent.reindex(new_dates)

    # fill null values
    petr_brent["VALUE (US$)"] = petr_brent["VALUE (US$)"].interpolate().bfill()

    # rename columns
    petr_brent.columns = ["price"]

    return petr_brent


# calculate descriptive statistics
def descriptive_statistics(petr_brent):

    # calculate null values
    # null_sum = petr_brent['price'].isnull().sum()
    # not_null_sum = len(petr_brent) - null_sum
    #
    # print(null_sum, not_null_sum)
    #
    # # calculate descriptive statistics
    df_describe = pd.DataFrame({"price": petr_brent.describe()["price"]}).reset_index()
    df_describe["index"] = [
        "Contagem",
        "Média (US$)",
        "Desvio Padrão (US$)",
        "Mínimo (US$)",
        "Quantil 25% (US$)",
        "Mediana (US$)",
        "Quantil 75% (US%)",
        "Máximo (US$)",
    ]

    return df_describe


# load exogenous data
def load_exog_data():

    change_ener_consump = pd.read_csv("data/change-energy-consumption.csv")
    fossil_fuel_consump = pd.read_csv("data/fossil-fuel-consumption-by-type.csv")
    fossil_fuel = pd.read_csv("data/fossil-fuel-primary-energy.csv")
    oil_prod_country = pd.read_csv("data/oil-production-by-country.csv")
    oil_share = pd.read_csv("data/oil-share-energy.csv")
    energy_percapita = pd.read_csv("data/per-capita-energy-use.csv")

    return (
        change_ener_consump,
        fossil_fuel_consump,
        fossil_fuel,
        oil_prod_country,
        oil_share,
        energy_percapita,
    )


if __name__ == "__main__":

    petr_brent = load_data()

    print(petr_brent.describe())

    df_describe = descriptive_statistics(petr_brent)

    print(df_describe)


def stationarity_test(series):
    series.dropna(inplace=True)
    result = adfuller(series)
    # print('ADF Statistic:', result[0])
    # print('p-value:', result[1])
    # for key, value in result[4].items():
    #     print('Critial Values:')
    #     print(f'   {key}, {value}')

    pvalue = result[1]

    results = []

    if pvalue > 0.05:
        results.append(pvalue)
        results.append("Não é estacionária.")
    else:
        results.append(pvalue)
        results.append("É estacionária.")

    return results  # return p-value


             price
count  8562.000000
mean     68.063615
std      28.648127
min       9.120000
25%      46.147500
50%      66.015000
75%      87.960000
max     143.950000
                 index        price
0             Contagem  8562.000000
1          Média (US$)    68.063615
2  Desvio Padrão (US$)    28.648127
3         Mínimo (US$)     9.120000
4    Quantil 25% (US$)    46.147500
5        Mediana (US$)    66.015000
6    Quantil 75% (US%)    87.960000
7         Máximo (US$)   143.950000


In [3]:
data = get_ipea_data()
(
    change_ener_consump,
    fossil_fuel_consump,
    fossil_fuel,
    oil_prod_country,
    oil_share,
    energy_percapita,
) = load_exog_data()


In [4]:
oil_prod_country_gt_2001 = oil_prod_country.query("Year > 2001")
oil_prod_country_gt_2001["Code"] = oil_prod_country_gt_2001["Code"].astype(str)
oil_prod_country_gt_2001 = oil_prod_country_gt_2001.query(
    "Code != 'nan' and Entity != 'World'"
)

fossil_fuel_gt_2001 = fossil_fuel.query("Year > 2001")

fossil_fuel_gt_2001["Code"] = fossil_fuel_gt_2001["Code"].astype(str)
fossil_fuel_gt_2001 = fossil_fuel_gt_2001.query("Code != 'nan' and Entity != 'World'")


/var/folders/kp/sr6lvhb16131kyr7jp9mwlyw0000gn/T/ipykernel_32881/434424578.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oil_prod_country_gt_2001['Code'] = oil_prod_country_gt_2001['Code'].astype(str)
/var/folders/kp/sr6lvhb16131kyr7jp9mwlyw0000gn/T/ipykernel_32881/434424578.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fossil_fuel_gt_2001['Code'] = fossil_fuel_gt_2001['Code'].astype(str)


In [5]:
change_ener_consump_gt_2001 = change_ener_consump.query("Year > 2001")


# limits the displayed country names by value
change_ener_consump_gt_2001["Code"] = change_ener_consump_gt_2001["Code"].astype(str)
change_ener_consump_gt_2001["Entity"] = change_ener_consump_gt_2001["Entity"].astype(
    str
)
change_ener_consump_gt_2001 = change_ener_consump_gt_2001.query(
    "Entity in ('United States','Brazil' ,'China', 'Japan', 'Germany', 'United Kingdom', \
    'Canada', 'India', 'France', 'Norway','South Africa') "
)


/var/folders/kp/sr6lvhb16131kyr7jp9mwlyw0000gn/T/ipykernel_32881/3027635014.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  change_ener_consump_gt_2001['Code'] = change_ener_consump_gt_2001['Code'].astype(str)
/var/folders/kp/sr6lvhb16131kyr7jp9mwlyw0000gn/T/ipykernel_32881/3027635014.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  change_ener_consump_gt_2001['Entity'] = change_ener_consump_gt_2001['Entity'].astype(str)


In [6]:
data.head()


,DATE,CODE,RAW DATE,DAY,MONTH,YEAR,VALUE (US$)
0,2001-01-01,EIA366_PBRENT366,2001-01-01T00:00:00-02:00,1,1,2001,NaN
1,2001-01-02,EIA366_PBRENT366,2001-01-02T00:00:00-02:00,2,1,2001,23.43
2,2001-01-03,EIA366_PBRENT366,2001-01-03T00:00:00-02:00,3,1,2001,23.44
3,2001-01-04,EIA366_PBRENT366,2001-01-04T00:00:00-02:00,4,1,2001,24.57
4,2001-01-05,EIA366_PBRENT366,2001-01-05T00:00:00-02:00,5,1,2001,24.77


In [8]:
# from daily to annual
price = data[["DATE", "VALUE (US$)"]].rename(
    columns={"DATE": "date", "VALUE (US$)": "price"}
)
price["date"] = pd.to_datetime(price["date"])
price_y = price.set_index("date").resample("Y").mean()

# price percent change
price_y_percent_change = price_y.resample("Y").sum().pct_change() * 100

# oil production data frame
df_production = oil_prod_country_gt_2001.reset_index()[
    ["Entity", "Year", "Oil production (TWh)"]
].pivot(index="Year", columns="Entity", values="Oil production (TWh)")

# price data frame
df_price = price_y.reset_index()
df_price["Year"] = df_price["date"].dt.year
df_price = df_price.drop(columns="date", axis=1)
df_price.set_index("Year", inplace=True)

# normalize data
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objs as go

# Normalizing the production data
scaler = MinMaxScaler()
normalized_production = df_production.copy()
normalized_production.iloc[:, 1:] = scaler.fit_transform(
    df_production.iloc[:, 1:].to_numpy()
)

# Normalizing the price data
normalized_price = df_price.copy().reset_index()
normalized_price.iloc[:, 1:] = scaler.fit_transform(normalized_price.iloc[:, 1:])
normalized_price.set_index("Year", inplace=True)


In [9]:
# consumption percent change
df_consump_pct_change = change_ener_consump_gt_2001[
    ["Year", "Entity", "Annual change in primary energy consumption (%)"]
].copy()
df_consump_pct_change.rename(
    columns={
        "Annual change in primary energy consumption (%)": "Percent Change",
        "Entity": "Type",
    },
    inplace=True,
)
df_consump_pct_change.set_index("Year", inplace=True)

# st.write(df_consump_pct_change.head())

# df_consump_melted = df_consump_pct_change.reset_index().melt(
#     id_vars=["Year"], var_name="Type",
#     value_vars= ["Entity", "Percent Change"],

# )

# st.write(df_consump_melted)

# price percent change
price_y_percent_change = price_y.resample("Y").sum().pct_change() * 100

# price data frame
df_price_pct_change = price_y_percent_change.reset_index()

df_price_pct_change["Year"] = df_price_pct_change["date"].dt.year
df_price_pct_change = df_price_pct_change.drop(columns="date", axis=1)

# prepare the price data frame to concatenate
df_price_melted = df_price_pct_change.reset_index().melt(
    id_vars=["Year"],
    var_name="Type",
    value_vars="price",
    value_name="Percent Change",
)
df_price_melted.set_index("Year", inplace=True)

# st.write(df_price_melted)

# Combine the concumption and price data for plotting
# df_combined = pd.concat([df_consump_pct_change, df_price_melted])
# df_combined["Type"] = df_combined["Type"].fillna(df_combined["Entity"])
# df_combined.drop(columns=["Entity"], inplace=True)
# df_combined.reset_index(inplace=True)
# Plot the data
# fig = px.line(df_combined, x="Year", y="Percent Change", color="Type")
# st.plotly_chart(fig)

df_consump_pct_change.reset_index(inplace=True)
# transform from long to wide format
df_consump_pct_change = df_consump_pct_change.pivot(
    index="Year", columns="Type", values="Percent Change"
)


In [15]:
x = df_price_pct_change[["Year", "price"]].set_index("Year").dropna()
# y = df_consump_pct_change['Canada'] #df_consump_pct_change.loc[:,'Canada']

# st.write(df_consump_pct_change)

granger_result = dict()
for country in df_consump_pct_change.columns[1:]:
    # st.write(country)
    # st.write(dash_utils.stationarity_test(df_consump_pct_change[country]))
    try:
        y = df_consump_pct_change[country]
        data = pd.merge(x, y, on="Year")
        # st.write(data)
        res = granger_test(data)
    except Exception as e:
        res = e
    # st.write(res)
    granger_result[country] = res


In [16]:
granger_result


{'Canada': ValueError('array length 20 does not match index length 21'),
 'China': ValueError('array length 20 does not match index length 21'),
 'France': ValueError('array length 20 does not match index length 21'),
 'Germany': KeyError("['index'] not found in axis"),
 'India': ValueError('array length 20 does not match index length 21'),
 'Japan': KeyError("['index'] not found in axis"),
 'Norway': KeyError("['index'] not found in axis"),
 'South Africa': ValueError('array length 20 does not match index length 21'),
 'United Kingdom': KeyError("['index'] not found in axis"),
 'United States': KeyError("['index'] not found in axis")}

In [17]:
y = df_consump_pct_change["Canada"]


In [18]:
x


,price
Year,
2002,3.223834
2003,14.297884
2004,32.261213
2005,42.355954
2006,19.863593
2007,10.616787
2008,35.403878
2009,-37.134152
2010,29.573968


In [19]:
y


Year
2002    3.395617
2003    0.477290
2004    1.537216
2005    0.668454
2006   -0.561678
2007    3.814864
2008   -1.189446
2009   -3.917259
2010    0.935554
2011    3.770900
2012    0.304627
2013    2.386391
2014    0.217402
2015    0.651622
2016   -1.359600
2017    1.522696
2018    0.902772
2019   -0.597560
2020   -6.022268
2021    0.944972
2022    2.206516
Name: Canada, dtype: float64

In [34]:
def granger_test(data):

    x = data.iloc[:, 0]
    y = data.iloc[:, 1]

    x_stationary = stationarity_test(x)
    y_stationary = stationarity_test(y)

    # if x_stationary[0] > 0.05:
    #     x = np.diff(x)
    #     print("x", x.shape)
    # if y_stationary[0] > 0.05:
    #     y = np.diff(y)
    #     print("y", y.shape)
    # Ensure there are no missing values
    # x = x[~np.isnan(x)]  # x = x
    # y = y[~np.isnan(y)]

    if x_stationary[0] > 0.05 or y_stationary[0] > 0.05:
        x = np.diff(x)
        y = np.diff(y)

    data = pd.DataFrame({"x": x, "y": y}, dtype=float)

    # Perform Granger causality test
    max_lag = 3
    granger_test_result = grangercausalitytests(
        data[["x", "y"]], maxlag=max_lag, verbose=False
    )
    # Initialize the result list
    result = []

    # Interpret the results
    for lag in range(1, max_lag + 1):
        test_results = granger_test_result[lag][0]
        min_pvalue = min(
            test_results["ssr_ftest"][1],
            test_results["ssr_chi2test"][1],
            test_results["lrtest"][1],
            test_results["params_ftest"][1],
        )
        if min_pvalue <= 0.05:
            causality_result = "granger causes"
        else:
            causality_result = "granger does not cause"

        result.append({lag: [min_pvalue, causality_result]})

    return result


In [35]:
df = pd.merge(x, y, on="Year")


In [44]:
res = dict()
res["Canada"] = granger_test(df)


/Users/carlosneves/Documents/Projects/fiap/fiap_techchallenge04/.venv/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [46]:
res["Canada"]


[{1: [0.8772530629949639, 'granger does not cause']},
 {2: [0.023123867103214103, 'granger causes']},
 {3: [0.05783266967583712, 'granger does not cause']}]